In [1]:
import pandas as pd
import cohere
import time
from cohere.responses.classify import Example
from tqdm import tqdm

In [2]:
api_key = '0KeTU61A6Yci4y3En9gGOxBpI6aHzmx5d5LSQVrc'
co = cohere.Client(api_key)

In [3]:
examples=[
  ("The order came 5 days early", 0.98), 
  ("The item exceeded my expectations", 0.95), 
  ("I ordered more for my friends", 0.8), 
  ("I would buy this again", 0.8), 
  ("I would recommend this to others", 0.8), 
  ("The package was damaged", -0.6), 
  ("The order is 5 days late", -0.98), 
  ("The order was incorrect", -0.8), 
  ("I want to return my item", -0.7), 
  ("The item\'s material feels very low quality", -0.9), 
  ("The product was okay", 0), 
  ("I received five items in total", 0), 
  ("I bought it from the website", 0), 
]

In [4]:
inputs=[
  "This item was broken when it arrived",
  "The electric bill is very high",
  "The food is awsome here"]

In [5]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereSentimentScore():
    
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        
        ## Define the prompt to extract entities from
        
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 str(labels[i]) for i in range(len(examples))]))

    def sentiments_score(self, example):
        # Call the entity extraction endpoint
        sentiment_score = co.generate(model='command',prompt=self.make_prompt(example),max_tokens=100,temperature=0.6,stop_sequences=["\n"])
        return(sentiment_score.generations[0].text)


cohere_sentiments_score = cohereSentimentScore([e[0] for e in examples], [e[1] for e in examples], [],"Below are examples of sentiment texts and their scores", "Sentiment Score:")


In [6]:
results = []
for text in tqdm(inputs):
    try:
        sentiments_scores = cohere_sentiments_score.sentiments_score(text)
        results.append(sentiments_scores.strip())
    except Exception as e:
        print('ERROR: ', e)
results

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.19s/it]


['-0.95', '-0.8', '0.9']

In [7]:
for i,j in zip(inputs,results):
    print(i,j)

This item was broken when it arrived -0.95
The electric bill is very high -0.8
The food is awsome here 0.9
